# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.5,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.5,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)
goal_col = gscene.create_safe(GEOTYPE.BOX, "goal_col", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.006), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=False, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.6,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.6,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.6,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.1,0.07,0.02), 
                   center=(0.3,-0.4,0.04), rpy=(0,0,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="base_link", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
# box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [17]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [19]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, end_link_couple_dict= {TOOL_LINK: TOOL_LINK_BUNDLE, "base_link":["base_link"]})
# checkers_all.append(lcheck)

In [20]:
mplan.motion_filters = checkers_all

## Test plan

In [21]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)

# Node Sampler

In [22]:
from pkg.p

In [36]:
tplan.new_node_sampler = NodeSampler(0.5)
tplan.parent_node_sampler = NodeSampler(0.5)

In [37]:
gtimer.reset()
# goal_nodes = [("goal",)]
# goal_nodes = [("goal", "floor", "floor", 2, 2, 2)]
goal_nodes = [("floor", "goal", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = success
branching: 0->1 (0.07/300.0 s, steps/err: 45(36.30900383 ms)/0.000887727738386)
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = success
branching: 0->2 (0.14/300.0 s, steps/err: 34(85.791110992

node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = success
branching: 1->19 (0.69/300.0 s, steps/err: 39(600.422143936 ms)/0.000711715966512)
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = success
branching: 3->20 (0.76/300.0 s, steps/err: 45(179.538011551 ms)/0.00156416593597)
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'goal', 0, 0, 0) = success
branching: 3->21 (0.82/300.0 s, steps/err: 32(125.292062759 ms)/0.00148304079234)
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 1, 0) = success
branching: 1->22 (0.88/300.0 s, steps/err: 39(31.5790176392 ms)/0.00165760156454)
node: ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = fail
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = success
branching: 7->23 (0.99/300.0 s, steps/err: 31(94.0511226654 ms)/0.00153092174968)
node: ('grip0', 'goal', 0, 0, 0)->('goal', 'goal', 0, 0, 0) = fail
node: ('grip0', 'goal'

node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 1, 0, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 1, 0, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 1) = fail
node: ('grip0', 'goal', 0, 0, 0)->('floor', 'goal', 0, 0, 0) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = success
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 1, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 0, 0, 1) = success
branching: 11->32 (1.73/300.0 s, steps/err: 97(1169.1839695 ms)/0.00199533509759)
branching: 1->33 (1.74/300.0 s, steps/err: 40(123.192071915 ms)/0.00187270956349)
node: ('floor', 'grip0', 0, 0, 2)->('floor', 'grip0', 0, 1, 2) = s

node: ('grip0', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
branching: 32->49 (2.13/300.0 s, steps/err: 41(260.163068771 ms)/0.00145807797196)
node: ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 0, 1, 2) = fail
node: ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 1, 0, 2) = fail
node: ('floor', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 0, 0) = fail
node: ('floor', 'grip0', 0, 0, 1)->('floor', 'grip0', 0, 0, 2) = fail
node: ('floor', 'floor', 0, 0, 2)->('grip0', 'floor', 0, 0, 2) = fail
node: ('floor', 'grip0', 0, 0, 2)->('floor', 'floor', 0, 0, 2) = success
node: ('floor', 'goal', 0, 0, 2)->('floor', 'goal', 0, 1, 2) = fail
branching: 32->50 (2.17/300.0 s, steps/err: 7(352.694034576 ms)/0.00142303292144)
node: ('floor', 'floor', 0, 0, 2)->('grip0', 'floor', 0, 0, 2) = fail
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 0, 1, 0) = fail
node: ('floor', 'floor', 0, 0, 2)->('grip0', 'floor', 0, 0, 2) = fail
node: ('floor', 'grip0', 0, 0, 2)->('floor', 'grip0', 0, 1, 2) = suc

branching: 32->65 (2.92/300.0 s, steps/err: 14(734.120130539 ms)/0.00166059550959)
node: ('floor', 'floor', 2, 0, 2)->('floor', 'floor', 2, 1, 2) = fail
node: ('goal', 'floor', 0, 0, 0)->('goal', 'grip0', 0, 0, 0) = fail
node: ('floor', 'grip0', 1, 0, 2)->('floor', 'grip0', 2, 0, 2) = fail
node: ('floor', 'goal', 0, 0, 2)->('floor', 'goal', 0, 1, 2) = fail
node: ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 1, 0, 2) = fail
node: ('floor', 'floor', 2, 0, 2)->('floor', 'floor', 2, 1, 2) = fail
node: ('floor', 'goal', 2, 0, 2)->('grip0', 'goal', 2, 0, 2) = fail
node: ('grip0', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = fail
node: ('goal', 'floor', 0, 0, 0)->('goal', 'floor', 0, 0, 1) = fail
node: ('floor', 'grip0', 2, 0, 2)->('floor', 'grip0', 2, 1, 2) = success
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 1, 0, 0) = fail
branching: 55->66 (2.99/300.0 s, steps/err: 10(189.443826675 ms)/0.00139051376913)
node: ('grip0', 'goal', 0, 0, 0)->('grip0', 'goal', 1, 0, 0) = fail
nod

...

node: ('goal', 'floor', 2, 2, 0)->('grip0', 'floor', 2, 2, 0) = fail
branching: 32->1955 (88.91/300.0 s, steps/err: 21(128.677845001 ms)/0.00179029375707)
node: ('goal', 'grip0', 1, 2, 2)->('goal', 'grip0', 2, 2, 2) = fail
node: ('goal', 'floor', 2, 0, 0)->('goal', 'floor', 2, 1, 0) = fail
node: ('goal', 'grip0', 0, 0, 2)->('goal', 'goal', 0, 0, 2) = fail
node: ('grip0', 'goal', 0, 0, 2)->('floor', 'goal', 0, 0, 2) = fail
node: ('goal', 'grip0', 0, 2, 0)->('goal', 'grip0', 0, 2, 1) = fail
node: ('goal', 'floor', 0, 0, 2)->('grip0', 'floor', 0, 0, 2) = fail
node: ('floor', 'grip0', 0, 0, 0)->('floor', 'grip0', 1, 0, 0) = fail
node: ('goal', 'grip0', 0, 0, 0)->('goal', 'grip0', 0, 0, 1) = success
branching: 661->1956 (88.98/300.0 s, steps/err: 2(31.0368537903 ms)/0.00158838905191)
node: ('goal', 'grip0', 2, 0, 0)->('goal', 'grip0', 2, 1, 0) = success
branching: 1453->1957 (89.0/300.0 s, steps/err: 36(50.9798526764 ms)/0.00142353344656)
node: ('goal', 'grip0', 0, 0, 0)->('goal', 'floor', 

branching: 1453->1969 (89.82/300.0 s, steps/err: 26(40.5728816986 ms)/0.00176022857144)
node: ('floor', 'goal', 2, 2, 0)->('floor', 'goal', 2, 2, 1) = fail
node: ('floor', 'goal', 2, 2, 0)->('floor', 'goal', 2, 2, 1) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
node: ('floor', 'floor', 0, 2, 0)->('floor', 'floor', 1, 2, 0) = fail
node: ('goal', 'floor', 0, 2, 0)->('goal', 'floor', 1, 2, 0) = fail
node: ('goal', 'grip0', 2, 0, 0)->('goal', 'grip0', 2, 1, 0) = success
branching: 94->1971 (89.89/300.0 s, steps/err: 27(50.6980419159 ms)/0.00187311709398)
node: ('floor', 'grip0', 2, 0, 2)->('floor', 'grip0', 2, 1, 2) = success
node: ('floor', 'grip0', 2, 0, 0)->('floor', 'floor', 2, 0, 0) = fail
branching: 1601->1972 (89.9/300.0 s, steps/err: 20(157.758951187 ms)/0.0010182796835)
node: ('grip0', 'floor', 0, 0, 2)->('floor', 'floor', 0, 0, 2) = fail
node: ('goal', 'floor', 2, 0, 0)->('goal', 'floor', 2, 1, 0) = fail
node: ('goal', 'grip0', 2, 2, 2)->('goal', '

node: ('goal', 'grip0', 2, 2, 1)->('goal', 'grip0', 2, 2, 2) = fail
node: ('goal', 'grip0', 0, 2, 1)->('goal', 'grip0', 0, 2, 2) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 1, 0) = fail
node: ('goal', 'grip0', 1, 0, 2)->('goal', 'grip0', 2, 0, 2) = fail
node: ('grip0', 'floor', 0, 0, 0)->('grip0', 'floor', 0, 0, 1) = fail
node: ('goal', 'grip0', 0, 2, 2)->('goal', 'goal', 0, 2, 2) = fail
node: ('floor', 'grip0', 0, 0, 2)->('floor', 'goal', 0, 0, 2) = success
node: ('floor', 'grip0', 2, 2, 0)->('floor', 'grip0', 2, 2, 1) = fail
node: ('goal', 'floor', 2, 0, 2)->('goal', 'floor', 2, 1, 2) = fail
branching: 713->1986 (90.98/300.0 s, steps/err: 6(30.7550430298 ms)/0.00183937055104)
node: ('goal', 'grip0', 0, 0, 0)->('goal', 'grip0', 0, 1, 0) = success
node: ('goal', 'grip0', 0, 2, 2)->('goal', 'floor', 0, 2, 2) = success
branching: 248->1987 (91.04/300.0 s, steps/err: 10(926.931858063 ms)/0.00183838559632)
branching: 1028->1988 (91.05/300.0 s, steps/err: 11(1820.4779624

node: ('goal', 'grip0', 0, 2, 2)->('goal', 'goal', 0, 2, 2) = fail
node: ('grip0', 'goal', 0, 0, 2)->('goal', 'goal', 0, 0, 2) = fail
node: ('goal', 'floor', 2, 0, 0)->('goal', 'floor', 2, 1, 0) = fail
node: ('goal', 'grip0', 1, 2, 0)->('goal', 'grip0', 2, 2, 0) = fail
node: ('grip0', 'floor', 2, 0, 2)->('grip0', 'floor', 2, 1, 2) = fail
node: ('floor', 'floor', 0, 0, 0)->('floor', 'grip0', 0, 0, 0) = success
branching: 1436->2002 (91.51/300.0 s, steps/err: 45(124.205112457 ms)/0.000507750220326)
node: ('floor', 'floor', 2, 2, 0)->('grip0', 'floor', 2, 2, 0) = fail
node: ('goal', 'grip0', 2, 2, 0)->('goal', 'grip0', 2, 2, 1) = success
branching: 510->2003 (91.51/300.0 s, steps/err: 35(110.994100571 ms)/0.00139773236559)
node: ('goal', 'grip0', 0, 2, 0)->('goal', 'goal', 0, 2, 0) = fail
node: ('grip0', 'goal', 2, 0, 0)->('grip0', 'goal', 2, 1, 0) = fail
node: ('grip0', 'floor', 2, 0, 0)->('goal', 'floor', 2, 0, 0) = fail
node: ('floor', 'floor', 0, 0, 2)->('floor', 'floor', 0, 1, 2) = f

node: ('goal', 'grip0', 1, 2, 2)->('goal', 'grip0', 2, 2, 2) = fail
======================= terminated 19: first answer acquired from other agent ===============================
node: ('goal', 'grip0', 0, 2, 1)->('goal', 'grip0', 0, 2, 2) = fail
======================= terminated 3: first answer acquired from other agent ===============================
node: ('goal', 'grip0', 0, 1, 0)->('goal', 'grip0', 0, 2, 0) = fail
======================= terminated 12: first answer acquired from other agent ===============================
node: ('floor', 'grip0', 1, 0, 0)->('floor', 'grip0', 2, 0, 0) = fail
======================= terminated 7: first answer acquired from other agent ===============================


In [38]:
print(gtimer)

plan: 	91687.0 ms/1 = 91686.716 ms (91686.716/91686.716)



## play searched plan

In [26]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [27]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [31]:
ppline.play_schedule(snode_schedule, period=0.03)

In [32]:
len(snode_schedule)

18

In [30]:
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

0: ('floor', 'floor', 0, 0, 0)
1: ('floor', 'grip0', 0, 0, 0)
2: ('floor', 'floor', 0, 0, 0)
3: ('grip0', 'floor', 0, 0, 0)
4: ('goal', 'floor', 0, 0, 0)
5: ('goal', 'grip0', 0, 0, 0)
6: ('goal', 'grip0', 0, 0, 1)
7: ('goal', 'grip0', 0, 0, 2)
8: ('goal', 'grip0', 0, 1, 2)
9: ('goal', 'grip0', 0, 2, 2)
10: ('goal', 'floor', 0, 2, 2)
11: ('grip0', 'floor', 0, 2, 2)
12: ('floor', 'floor', 0, 2, 2)
13: ('floor', 'grip0', 0, 2, 2)
14: ('floor', 'grip0', 1, 2, 2)
15: ('floor', 'grip0', 2, 2, 2)
16: ('floor', 'goal', 2, 2, 2)
17: ('floor', 'goal', 2, 2, 2)


## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가해보기?

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}